# Actividad: Match filter



In [ ]:
%matplotlib notebook
import numpy as np
import scipy.signal
import matplotlib.pylab as plt
from matplotlib import animation
from IPython.display import Audio

In [ ]:
def sine_pulse(n, k0, center):
    x = np.zeros_like(n, dtype=np.float); L = int(1/k0)
    x[center:center+L] = np.sin(2.0*np.pi*k0*(n[center:center+L] - n[center]))
    return x

fig, ax = plt.subplots(3, figsize=(8, 6), tight_layout=True)
np.random.seed(0); 
n = np.arange(0, 100, step=1)
x_clean = sine_pulse(n, k0=0.05, center=50) 
ax[0].plot(n, x_clean)
s = 1.;  x = x_clean + s*np.random.randn(len(n))
ax[0].plot(n, x, 'k.'); ylims = ax[0].get_ylim()
ax[0].set_title("SNR %0.2f [dB]" %(10*np.log(0.5/s**2)))
# System:
h = np.sin(2.0*np.pi*0.05*n[:int(len(n)/5)] + np.pi); h = h/np.sum(np.absolute(h))
# Acumulator
y = np.zeros_like(n, dtype=np.float)
def init():
    global y
    y = np.zeros_like(n, dtype=np.float)
    return ()
    
def update(m):
    c = np.zeros_like(n, dtype=np.float); c[m:m+len(h)] = h
    ax[1].cla(); ax[1].plot(c); 
    y[m] = np.sum(h*x[m:m+len(h)])
    ax[2].cla(); ax[2].plot(n, y**2);  #ax[2].set_ylim(ylims)
    ax[2].plot([m, m], [np.amin(y**2), np.amax(y**2)], 'r--', alpha=0.5)
    return ()

anim = animation.FuncAnimation(fig, update, init_func=init, 
                               frames=100-len(h), interval=200, blit=True)